# 📘 Modelo de Optimización de Producción de Wafers
**Hackathon Talent Land - Micron | Abril 2025**

Este notebook presenta la formulación matemática completa del modelo de programación lineal (PL) para optimizar la planificación de producción de wafers cumpliendo con restricciones de inventario, demanda, capacidad y prioridad.

## 🔢 Variables de Decisión

- $x_{i,t} \in \mathbb{Z}_+$: Producción del producto `i` en la semana `t`.
- $S_{i,t} \in \mathbb{R}_+$: Inventario final del producto `i` al final de la semana `t`.

Donde:
- $i \in \{21A, 22B, 23C\}$
- $t \in \{1, 2, ..., T\}$

## 🎯 Función Objetivo

Minimizar los costos totales de producción y almacenamiento:

\[ \min \sum_{i} \sum_{t} \left( C_{i,t} \cdot x_{i,t} + H_{i,t} \cdot S_{i,t} \right) \]

## 📏 Restricciones

### 1. Balance de Inventario
\[ S_{i,t} = \begin{cases} S_{i,0} + x_{i,1} - D_{i,1}, & t = 1 \\ S_{i,t-1} + x_{i,t} - D_{i,t}, & t > 1 \end{cases} \]

### 2. Producción mínima total semanal
\[ \sum_{i} x_{i,t} \geq 350, \quad \forall t \]

### 3. Ramp-up máximo
\[ x_{i,t} - x_{i,t-1} \leq 560, \quad \forall i, t > 1 \]

### 4. Producción congelada
\[ x_{i,t} = \bar{x}_{i,t}, \quad \forall i, t \in T_{\text{congelado}} \]

### 5. Producción en múltiplos de 5
\[ x_{i,t} = 5 \cdot z_{i,t}, \quad z_{i,t} \in \mathbb{Z}_+ \]

### 6. Prioridad de demanda
\[ x_{21A,t} \geq D_{21A,t} \]
\[ x_{22B,t} + \text{Exceso}_{21A,t} \geq D_{22B,t} \]
\[ x_{23C,t} + \text{Exceso}_{22B,t} \geq D_{23C,t} \]

### 7. Inventario mínimo
\[ S_{i,t} \geq SST_i, \quad \forall i, t \]

### 8. Rango de inventario excedente trimestral
\[ 70M \leq \sum_{i} \sum_{t \in Q} (S_{i,t} - SST_i) \leq 140M \]

In [ ]:
# Cargar datos desde archivo Excel
import pandas as pd

file_path = "../data/Hackaton DB Final 04.21.xlsx"
demand_df = pd.read_excel(file_path, sheet_name="Demand")
inventory_df = pd.read_excel(file_path, sheet_name="Inventory")
yield_df = pd.read_excel(file_path, sheet_name="Yield")

demand_df.head()

In [ ]:
# Visualizar demanda por producto
import matplotlib.pyplot as plt

for product in demand_df["Product"].unique():
    df = demand_df[demand_df["Product"] == product]
    plt.plot(df["Week"], df["Demand"], label=f"Demand {product}")

plt.legend()
plt.title("Demanda por producto")
plt.xlabel("Semana")
plt.ylabel("Demanda")
plt.grid(True)
plt.show()

## ⚙️ Definición del Modelo con PuLP

In [ ]:
from pulp import *

# Productos y semanas de ejemplo (pueden ser reemplazados por lectura dinámica)
products = ['21A', '22B', '23C']
weeks = list(range(1, 14))  # 13 semanas

# Parámetros ficticios para pruebas (reemplazar con datos reales)
demand = {(p, w): 100 for p in products for w in weeks}
initial_inventory = {p: 50 for p in products}
sst = {p: 40 for p in products}
max_ramp_up = 560

# Modelo
model = LpProblem("Wafer_Production_Optimization", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("Production", (products, weeks), lowBound=0, cat='Integer')
s = LpVariable.dicts("Inventory", (products, weeks), lowBound=0)

# Objetivo (sin costos reales, solo como ejemplo)
model += lpSum(x[i][t] + 0.1 * s[i][t] for i in products for t in weeks)

# Restricciones de balance de inventario
for i in products:
    for t in weeks:
        if t == 1:
            model += s[i][t] == initial_inventory[i] + x[i][t] - demand[(i, t)]
        else:
            model += s[i][t] == s[i][t-1] + x[i][t] - demand[(i, t)]

# SST mínima
for i in products:
    for t in weeks:
        model += s[i][t] >= sst[i]

# Ramp-up
for i in products:
    for t in weeks[1:]:
        model += x[i][t] - x[i][t-1] <= max_ramp_up

# Producción total mínima por semana
for t in weeks:
    model += lpSum(x[i][t] for i in products) >= 350


## 🚀 Resolución del Modelo y Resultados

In [ ]:
# Resolver
model.solve()

# Mostrar producción e inventario resultantes
for i in products:
    print(f"Producto {i}")
    for t in weeks:
        print(f"  Semana {t}: Producción = {x[i][t].varValue}, Inventario = {s[i][t].varValue}")
